# Week 3: Improve MNIST with Convolutions

In the videos you looked at how you would improve Fashion MNIST using Convolutions. For this exercise see if you can improve MNIST to 99.5% accuracy or more by adding only a single convolutional layer and a single MaxPooling 2D layer to the model from the  assignment of the previous week. 

You should stop training once the accuracy goes above this amount. It should happen in less than 10 epochs, so it's ok to hard code the number of epochs for training, but your training must end once it hits the above metric. If it doesn't, then you'll need to redesign your callback.

When 99.5% accuracy has been hit, you should print out the string "Reached 99.5% accuracy so cancelling training!"


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [4]:
# load the data
current_dir = os.getcwd()

data_path = os.path.join(current_dir, 'data/mnist.npz')

(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data(path=data_path)

print(training_images.shape)

(60000, 28, 28)


In [5]:
def reshape_and_normalize(images):
    images = np.reshape(images, (60000, 28, 28, 1))
    
    images = images/255.0
    
    return images

In [7]:
# check the image after reshape and normalize

#load again in case you run the cell multiple times
(training_images, _), _ = tf.keras.datasets.mnist.load_data(path=data_path)

training_images = reshape_and_normalize(training_images)

print(f'Maximum pixels after normalizing images {np.max(training_images)}')
print(f'THe size of training image after reshaping {training_images.shape}')
print(f'The size of each image after reshaping {training_images[0].shape}')

Maximum pixels after normalizing images 1.0
THe size of training image after reshaping (60000, 28, 28, 1)
The size of each image after reshaping (28, 28, 1)


In [8]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.995):
            print("\nReached 99.5% accuracy so cancelling training!")
            self.model.stop_training = True

In [20]:
def convolution_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2,2)),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [22]:
model = convolution_model()

callbacks = myCallback()

history = model.fit(training_images, training_labels, epochs=10, callbacks=callbacks)

Epoch 1/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1336 - accuracy: 0.9605
Epoch 2/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0464 - accuracy: 0.9855
Epoch 3/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0291 - accuracy: 0.9910
Epoch 4/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0186 - accuracy: 0.9938
Epoch 5/10
1873/1875 [============================>.] - ETA: 0s - loss: 0.0131 - accuracy: 0.9958
Reached 99.5% accuracy so cancelling training!
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0131 - accuracy: 0.9958


In [28]:
print(f'Accuracy reached after training for {len(history.epoch)} epochs')

Accuracy reached after training for 5 epochs
